# Reviews (Publish)

In this section, we are going to create a couple of views that SnowBearAir dashboard developers will use to drive an executive dashboard for sentiments.

In particular, we will create:

* `AVG_OVERALL_SCORE_VW` : This details how SnowBearAir is doing, compared to our peers, in terms of overall scores submitted with reviews.
* `LAST_TWENTY_SENTIMENT_VW` : This will take the most recent reviews, and using our sentiment analysis UDF, provide some additional data and ▁▆██▅█▄.

Our goal is to use Snowpark, along with the other UDFs we have created (`ANALYZE_TEXT`, and `TINY_BARS`), to build these views suitable for publishing in our dashboard.

- [ ] Build a DataFrame for Overall Scores
- [ ] Publish as `AVG_OVERALL_SCORE_VW`
- [ ] Build a DataFrame and process sentiment, and visualize sentence sentiments
- [ ] Publish as `LAST_TWENTY_SENTIMENT_VW`

![](../assets/reviews_publish.gif)



In [ ]:
import com.snowflake.snowpark._
import com.snowflake.snowpark.functions._
import com.snowflake.snowpark.types._

In [ ]:
// Set connection properties built in de_snowpark/A-Dataframes/01-Sessions.ipynb
val pwd = sys.env.get("PWD").fold("")(_.toString)
val filename = s"$pwd/de_snowpark/connect.properties"

val session = Session.builder.configFile(s"$filename").create

// Set schema for this session to MODELED
session.sql("use schema MODELED").collect

### Average Overall Score by Airline

First, let's load the view created in the previous lab (note the use of the table() method to load views) and `transform` this data with aggregations to return a new DataFrame: grouping by airline, computing mean overall_rating, and sorting the result.

In [ ]:
val overallScoreByAirlineDF = session
    .table("CONFORMED.CLEAN_REVIEWS_VW")
    .groupBy(col("airline"))
    .agg(mean(col("OVERALL")).as("OVERALL_RATING"))
    .sort(col("OVERALL_RATING").desc)

In [ ]:
overallScoreByAirlineDF.show(20)

### Progress Check

- [X] Build a DataFrame for Overall Scores
- [ ] Publish as `AVG_OVERALL_SCORE_VW`
- [ ] Build a DataFrame and process sentiment, and visualize sentence sentiments
- [ ] Publish as `LAST_TWENTY_SENTIMENT_VW`

To generate a (permanent) view based on our DataFrame we invoke the following method:

In [ ]:
overallScoreByAirlineDF
    .createOrReplaceView("modeled.AVG_OVERALL_SCORE_VW")

<div class="alert alert-block alert-info">
<i class="fas fa-chart-line fa-2x"></i>
    Head over to <a href="https://app.snowflake.com/">app.snowflake.com</a> and login to the class Snowflake account using your animal name and password. Play around with and visualize the data available in the new view. What is the top airline in terms of overall customer score?
</div>

### Progress Check

- [X] Build a DataFrame for Overall Scores
- [X] Publish as `AVG_OVERALL_SCORE_VW`
- [ ] Build a DataFrame and process sentiment, and visualize sentence sentiments
- [ ] Publish as `LAST_TWENTY_SENTIMENT_VW`

# Sentiment and ▁▆██▅█▄ 

Now let's generate a new DataFrame, which again calls on the view we created in the previous lab. Filter on records for `SnowBearAir`, and add an additional column passing in the customer_review text to the sentiment analysis UDF created earlier. Visualize the result in a new DataFrame which also includes the `TINY_BAR` graphical output based on the input data.

In [ ]:
val sentimentLastTwentyReviewsDF = session
    .table("CONFORMED.CLEAN_REVIEWS_VW")
    .filter(col("AIRLINE") === "SnowBearAir")
    .sort(col("REVIEW_DATE_TYPED").desc)
    .limit(20)
    .withColumn("SENTIMENT", callUDF("RAW.ANALYZE_TEXT", substring(col("CUSTOMER_REVIEW"), lit(1), lit(200))))

In [ ]:
sentimentLastTwentyReviewsDF.show()

In [ ]:
val sentWithBarsDF = sentimentLastTwentyReviewsDF
    .select(Seq(
        col("REVIEW_DATE_TYPED").as("REVIEW_DATE")
        ,as_double(col("SENTIMENT")("AVERAGE_SENTIMENT")).as("AVG_SENTIMENT")
        ,col("CUSTOMER_REVIEW")
        ,as_array(col("SENTIMENT")("SENTIMENTS")).as("SENTIMENT_INTS")
    ))
    .withColumn("SENTIMENT_HISTOGRAM", callUDF("MODELED.TINY_BARS", col("SENTIMENT_INTS")))
    .select(Seq(
        col("REVIEW_DATE")
        ,col("SENTIMENT_HISTOGRAM")
        ,col("AVG_SENTIMENT")
        ,col("CUSTOMER_REVIEW")
    ))

In [ ]:
sentWithBarsDF.show(20)

### Progress Check

- [X] Build a DataFrame for Overall Scores
- [X] Publish as `AVG_OVERALL_SCORE_VW`
- [X] Build a DataFrame and process sentiment, and visualize sentence sentiments
- [ ] Publish as `LAST_TWENTY_SENTIMENT_VW`

Finally, let's generate a (permanent) view based on our DataFrame to publish our output:

In [ ]:
sentWithBarsDF
    .createOrReplaceView("modeled.LAST_TWENTY_SENTIMENT_VW")

### Progress Check

- [X] Build a DataFrame for Overall Scores
- [X] Publish as `AVG_OVERALL_SCORE_VW`
- [X] Build a DataFrame and process sentiment, and visualize sentence sentiments
- [X] Publish as `LAST_TWENTY_SENTIMENT_VW`

## You Try It

Navigate to <a href="https://app.snowflake.com/">app.snowflake.com</a> again and and create the most interesting dashboard you can, using any of the objects/data/views we have created.


![](../assets/reviews_model_snowsight.png)